<a href="https://colab.research.google.com/github/FacuML/NLP/blob/main/002_Parte/009/001_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 Aplicaciones prácticas de Transformers con Hugging Face

En esta clase vamos a usar modelos preentrenados en español para resolver tareas reales de Procesamiento del Lenguaje Natural (PLN), sin necesidad de entrenar modelos desde cero.

Trabajaremos con la librería `transformers` de Hugging Face, que permite usar modelos de tipo BERT, GPT y similares en muy pocas líneas de código.

---


In [ ]:
# Instalación de Hugging Face Transformers (solo una vez)
!pip install -q transformers

## 1. Cargando el pipeline de Hugging Face

Hugging Face proporciona "pipelines" que encapsulan todo el proceso: tokenización, modelo y decodificación. Solo tenés que indicar qué tarea querés hacer.


In [ ]:
from transformers import pipeline

## 2. Análisis de sentimiento en español

Vamos a usar un modelo entrenado para identificar si una frase expresa un **sentimiento positivo o negativo**. Este modelo fue entrenado con tweets en español.

In [ ]:
sentiment = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/481k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
frases = [
    "Este lugar está buenísimo, la atención de diez",
    "Una experiencia horrible, me quiero ir",
    "Zafa, pero esperaba más",
    "Recomiendo totalmente este producto",
    "Nunca más compro acá"
]

for frase in frases:
    print(f"{frase} → {sentiment(frase)}")

Este lugar está buenísimo, la atención de diez → [{'label': 'POS', 'score': 0.9988070726394653}]
Una experiencia horrible, me quiero ir → [{'label': 'NEG', 'score': 0.9991808533668518}]
Zafa, pero esperaba más → [{'label': 'NEG', 'score': 0.9256047010421753}]
Recomiendo totalmente este producto → [{'label': 'POS', 'score': 0.9985804557800293}]
Nunca más compro acá → [{'label': 'NEG', 'score': 0.9992826581001282}]


## 3. Clasificación de texto por tema (*zero-shot*)

¿Querés clasificar frases por categorías sin entrenar un modelo? ¡Esto es posible gracias al aprendizaje **zero-shot**!

El modelo puede asociar un texto con una o más **etiquetas** sugeridas por vos, aunque nunca fue entrenado específicamente para esas clases.

In [ ]:
classifier = pipeline("zero-shot-classification", model="Recognai/bert-base-spanish-wwm-cased-xnli")

config.json:   0%|          | 0.00/834 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
texto = "Lionel Messi firmó contrato con el Inter Miami y debutará esta semana."

etiquetas = ["deportes", "economía", "política", "espectáculos"]

print(classifier(texto, candidate_labels=etiquetas))

{'sequence': 'Lionel Messi firmó contrato con el Inter Miami y debutará esta semana.', 'labels': ['deportes', 'espectáculos', 'política', 'economía'], 'scores': [0.516789436340332, 0.2758041322231293, 0.1318969577550888, 0.0755094587802887]}


## 4. Resumen automático de textos

Este pipeline toma un texto largo y genera un resumen breve en español. Ideal para noticias, informes o textos descriptivos.

Usamos un modelo BERT2BERT adaptado al español.

In [ ]:
summarizer = pipeline(
    "summarization",
    model="csebuetnlp/mT5_multilingual_XLSum",
    tokenizer="csebuetnlp/mT5_multilingual_XLSum"
)

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


In [ ]:
parrafo = """
El Ministerio de Salud confirmó hoy que se ha logrado una reducción sostenida de casos de dengue en las últimas semanas.
Las campañas de prevención, sumadas a la llegada del frío, habrían contribuido a esta baja. Sin embargo, se pide a la población mantener las precauciones.
"""

resumen = summarizer(parrafo, max_length=50, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])


El número de casos de dengue en Estados Unidos alcanzó un nuevo récord.


In [ ]:
texto = """
La inflación en Argentina ha mostrado una leve desaceleración en el último mes, según el informe del INDEC.
Sin embargo, los analistas advierten que la tendencia aún no se revierte, y que podrían esperarse aumentos para el próximo trimestre.
"""

resumen = summarizer(texto, max_length=60, min_length=25, do_sample=False)
print(resumen[0]['summary_text'])


## 5. Traducción automática (Español → Inglés)

También podemos usar modelos preentrenados para **traducir textos**. En este caso, usaremos uno especializado para traducir del español al inglés.


In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

In [ ]:
texto = "La inteligencia artificial está cambiando el mundo."

print(translator(texto)[0]['translation_text'])


## 6. Generación de texto en español (GPT)

Con un modelo tipo GPT entrenado en español, podemos **generar texto a partir de un inicio dado**. Ideal para escribir contenido creativo, continuar frases, etc.


In [ ]:
generator = pipeline("text-generation", model="PlanTL-GOB-ES/gpt2-base-bne")

In [ ]:
prompt = "Un dinosaurio está"

resultado = generator(prompt, max_length=50, num_return_sequences=1)
print(resultado[0]['generated_text'])


## 7. Reflexión y discusión

- ¿Cuál de estos pipelines te pareció más sorprendente o útil?
- ¿Creés que estas herramientas podrían usarse en un proyecto real? ¿En cuál?
- ¿Notaste errores o sesgos? ¿Por qué creés que aparecen?



## 8. Actividad libre (opcional si hay tiempo)

Explorá uno de los pipelines y diseñá tu propio experimento:

- Probá frases con sarcasmo o jergas locales.
- Resumí un artículo de Wikipedia.
- Traducí algo complejo (tecnológico, poético, etc.).
- Completá una frase usando estilo formal o informal.

Al final compartimos los hallazgos más interesantes con el grupo 👀

